In [26]:
using DataStructures
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using ViewerGL
GL = ViewerGL
using BenchmarkTools
using LinearAlgebra

## FUNZIONE DA OTTIMIZZARE

In [27]:
function simplifyCells(V,CV)
    PRECISION = 5
    vertDict = DefaultDict{Array{Float64,1}, Int64}(0)
    index = 0
    W = Array{Float64,1}[]
    FW = Array{Int64,1}[]
    
    for incell in CV
        outcell = Int64[]
        for v in incell
         vert = V[:,v]
         key = map(approxVal(PRECISION), vert)
            if vertDict[key]==0
                index += 1
                vertDict[key] = index
                push!(outcell, index)
                push!(W,key)
            else
                push!(outcell, vertDict[key])
            end
        end
         append!(FW, [[Set(outcell)...]])  #posso sostituirlo con push!(FW, [Set(outcell)...])
    end
    return hcat(W...),FW
end

simplifyCells (generic function with 1 method)

#### Funzione di supporto a simplifyCells

In [28]:
function approxVal(PRECISION)
    function approxVal0(value)
        out = round(value, digits=PRECISION)
        if out == -0.0
            out = 0.0
        end
        return out
    end
    return approxVal0
end

approxVal (generic function with 1 method)

-----------------------------------------------------------------------------------------------------------

### Dati sottostanti utilizzati per test !!!

In [29]:
#devo capire bene cosa faccia simplifyCells per poterla ottimizzare e testare  . 

A=[1.0 2.1234567 3.0 4.0 5.0 6.0 7.0; 1.0 2.1234567 3.0 4.0 5.0 6.0 7.0]
B=[1,2,7,3,1]
simplifyCells(A,B)

([1.0 2.12346 7.0 3.0; 1.0 2.12346 7.0 3.0], [[1], [2], [3], [4], [1]])

Ho fatto una piccola stampa per capire cosa facesse la funzione simplifyCells , e ne deduco che appunto trova e rimuove le celle duplicate o quelle che contengono errori settando una PRECISION = 5 come posso vedere verranno prese in considerazione solamente 5 valori dopo il . , e come possiamo ancora vedere NON vengono prese in considerazione piu celle con lo stesso valore , ma riportata solamente una .

### Dati di input

In [30]:
V = [1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 13.0 14.0 15.0 16.0 17.0 18.0 19.0 20.0 21.0 22.0 23.0 24.0 25.0 26.0 27.0 28.0 29.0 30.0 31.0 32.0 33.0 34.0 35.0 36.0 37.0 38.0 39.0 40.0 41.0 42.0 43.0 44.0 45.0 46.0 47.0 48.0 49.0 50.0 51.0 52.0 53.0 54.0 55.0 56.0 57.0 58.0 59.0 60.0 61.0 62.0 63.0 64.0 65.0 66.0 67.0 68.0 69.0 70.0 71.0 72.0 73.0 74.0 75.0 76.0 77.0 78.0 79.0 80.0 81.0 82.0 83.0 84.0 85.0 86.0 87.0 88.0 89.0 90.0 91.0 92.0 93.0 94.0 95.0 96.0 97.0 98.0 99.0 100.0 101.0 102.0 103.0 104.0 105.0 106.0 107.0 108.0 109.0 110.0 111.0 112.0 113.0 114.0 115.0 116.0 117.0 118.0 119.0 120.0 121.0 122.0 123.0 124.0 125.0 126.0 127.0 128.0 129.0;
     1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 13.0 14.0 15.0 16.0 17.0 18.0 19.0 20.0 21.0 22.0 23.0 24.0 25.0 26.0 27.0 28.0 29.0 30.0 31.0 32.0 33.0 34.0 35.0 36.0 37.0 38.0 39.0 40.0 41.0 42.0 43.0 44.0 45.0 46.0 47.0 48.0 49.0 50.0 51.0 52.0 53.0 54.0 55.0 56.0 57.0 58.0 59.0 60.0 61.0 62.0 63.0 64.0 65.0 66.0 67.0 68.0 69.0 70.0 71.0 72.0 73.0 74.0 75.0 76.0 77.0 78.0 79.0 80.0 81.0 82.0 83.0 84.0 85.0 86.0 87.0 88.0 89.0 90.0 91.0 92.0 93.0 94.0 95.0 96.0 97.0 98.0 99.0 100.0 101.0 102.0 103.0 104.0 105.0 106.0 107.0 108.0 109.0 110.0 111.0 112.0 113.0 114.0 115.0 116.0 117.0 118.0 119.0 120.0 121.0 122.0 123.0 124.0 125.0 126.0 127.0 128.0 129.0]

CV = [[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11],[12],[13],[14],[15],[16],[17],[18],[19],[20],[21],[22],[23],[24],[25],[26],[27],[28],[29],[30],[31],[32],[33],[34],[35],[36],[37],[38],[39],[40],[41],[42],[43],[44],[45],[46],[47],[48],[49],[50],[51],[52],[53],[54],[55],[56],[57],[58],[59],[60],[13],[1],[129],[127],[111],[2],[2],[67],[20],[12],[66],[67],[25],[3],[62]]

#CV = [[1,2],[3,4],[5,6],[7,8],[9,10],[11,12],[13,14],[15,16],[17,18],[19,20],[21,22],[23,24],[25,26],[27,28],[29,30],[31,32],[33,34],[35,36],[37,38],[39,40],[41,42],[43,44],[45,46],[47,48],[49,50],[51,52],[53,54],[55,56],[57,58],[59,60],[61,62],[63,64],[65,66],[67,68],[69,70],[71,72],[73,74],[75,76],[77,78],[79,80],[81,82],[83,84],[85,86],[87,88],[89,90],[91,92],[93,94],[95,96],[97,98],[99,100],[101,102],[103,104],[105,106],[107,108],[109,110],[111,112],[113,114],[115,116],[117,118],[119,120]]

75-element Array{Array{Int64,1},1}:
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]
 [8]
 [9]
 [10]
 [11]
 [12]
 [13]
 ⋮
 [127]
 [111]
 [2]
 [2]
 [67]
 [20]
 [12]
 [66]
 [67]
 [25]
 [3]
 [62]

## Benchmark funzione originaria 

In [31]:
@btime simplifyCells(V,CV)

  65.531 μs (930 allocations: 88.97 KiB)


([1.0 2.0 … 66.0 62.0; 1.0 2.0 … 66.0 62.0], [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]  …  [2], [2], [64], [20], [12], [65], [64], [25], [3], [66]])

-----------------------------------------------------------------------------------------------------------------

## Controllo type unstable

In [59]:
@code_warntype simplifyCells(V,CV)

Variables
  #self#::Core.Compiler.Const(simplifyCells, false)
  V::Array{Float64,2}
  CV::Array{Array{Int64,1},1}
  PRECISION::Int64
  vertDict::DefaultDict{Array{Float64,1},Int64,Int64}
  index::Int64
  W::Array{Array{Float64,1},1}
  FW::Array{Array{Int64,1},1}
  @_9::Union{Nothing, Tuple{Array{Int64,1},Int64}}
  incell::Array{Int64,1}
  outcell::Array{Int64,1}
  @_12::Union{Nothing, Tuple{Int64,Int64}}
  v::Int64
  vert::Array{Float64,1}
  key::Array{Float64,1}

Body::Tuple{Array,Array{Array{Int64,1},1}}
1 ──       (PRECISION = 5)
│    %2  = Core.apply_type(Main.Array, Main.Float64, 1)::Core.Compiler.Const(Array{Float64,1}, false)
│    %3  = Core.apply_type(Main.DefaultDict, %2, Main.Int64)::Core.Compiler.Const(DefaultDict{Array{Float64,1},Int64,F} where F, false)
│          (vertDict = (%3)(0))
│          (index = 0)
│    %6  = Core.apply_type(Main.Array, Main.Float64, 1)::Core.Compiler.Const(Array{Float64,1}, false)
│          (W = Base.getindex(%6))
│    %8  = Core.apply_type(Main

Come possiamo vedere dall'output la funzione NON è di tipo unstable , possiamo notarlo grazie alla seguente riga : Body::Tuple{Array,Array{Array{Int64,1},1}} 

### Ottimizzazione con macro @inbounds 

Possiamo notare un leggerissimo miglioramento con la macro @inbounds che dice a Julia di saltare il controllo dei limiti di array nei cicli for .. ho provato ad aggiungerelo anche nel secondo for ma le prestazioni peggioravano , ed inoltre ho utilizzato anche la macro @simd ma non ha portato nessun beneficio .

In [22]:
 function simplifyCellsDUE(V,CV)
    PRECISION = 5
    vertDict = DefaultDict{Array{Float64,1}, Int64}(0)
    index = 0
    W = Array{Float64,1}[]
    FW = Array{Int64,1}[]
     
     @inbounds for incell in CV
       outcell = Int64[]
          for v in incell
           vert = V[:,v]
            key = map(approxVal(PRECISION), vert)
                if vertDict[key]==0
                 index += 1
                 @inbounds vertDict[key] = index
                push!(outcell, index)
                push!(W,key)
            else
                push!(outcell, vertDict[key])
            end
        end
        append!(FW, [[Set(outcell)...]]) 
    end
     return hcat(W...),FW
end

simplifyCellsDUE (generic function with 1 method)

In [23]:
@btime simplifyCellsDUE(V,CV)

  66.022 μs (930 allocations: 88.97 KiB)


([1.0 2.0 … 66.0 62.0; 1.0 2.0 … 66.0 62.0], [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]  …  [2], [2], [64], [20], [12], [65], [64], [25], [3], [66]])

---------------------------------------------------------------------------------------------------------------

## Uso della macro @threads 

#### Per settare il numero di threads pari a 2 ho dovuto per forza eseguire (come scritto nella guida di julia) i seguenti passaggi :  export JULIA_NUM_THREADS=2  , set JULIA_NUM_THREADS=2 , il numero di threads va settato in base alla macchina detenuta , se non lo si fa l'uso dei threads è nullo .

Procedura utilizzata per applicare la macro @threads : 

1) Importare libreria e definito numero di threads concorrenziali. Ho settato il numero di core della mia macchina attraverso Julia.
2) Individuo il ciclo for che ha più elementi su cui iterare. 
3) Individuare quali operazioni di scrittura o lettura le istruzioni dentro il ciclo for compiono rispetto a variabili esterne al ciclo for , se sono solo di lettura posso parallelizzare , se invece il for scrive su delle variabili esterne ad esso dovrò fare in modo che i vari thread siano sicuri al 100% di non scrivere per sbaglio sopra spazi di memoria appena scritti da altri thread concorrenziali prima di parallelizzare.


In [9]:
using Base.Threads
Threads.nthreads() = 2  #setto il numero di threads pari a 2

In [10]:
nthreads()

2

In [13]:
function simplifyCellsThreads(V,CV)
    PRECISION = 5
    vertDict = DefaultDict{Array{Float64,1}, Int64}(0)
    index = 0
    W = Array{Float64,1}[]
    FW = Array{Int64,1}[]
       
    @threads for incell in CV
                outcell= Int64[]
                for v in incell
             vert = V[:,v]
             key = map(approxVal(PRECISION), vert)
             if vertDict[key]==0
                 index += 1
                 vertDict[key] = index
                 push!(outcell, index)
                 push!(W,key)
             else
                 push!(outcell, vertDict[key])
             end
         end
         append!(FW, [[Set(outcell)...]]) 
     end
     return hcat(W...),FW
 end
        

simplifyCellsThreads (generic function with 1 method)

In [14]:
@btime simplifyCellsThreads(V,CV)

  70.493 μs (1000 allocations: 91.17 KiB)


([1.0 2.0 … 66.0 62.0; 1.0 2.0 … 66.0 62.0], [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]  …  [2], [2], [64], [20], [12], [65], [64], [25], [3], [66]])

L'uso dei threads nel mio caso aumenta il tempo di esecuzione , tutto dipende dalla macchina sui cui gira il codice , io non avendo una macchina molto potente ho settato il numero di threads pari a 2 ... ma gira ugualmente lento. I cicli for sono di fondamentale importanza per il giusto funzionamento dei threads e dovrebbero essere della forma incell=1:length(CV) , ovviamente in questo caso il tempo di esecuzione diminuisce ma l'output poi non sarà quello ! In effetti il tempo di esecuzione aumenta anche a causa del numero di allocazioni che sono aumentate.

## Refactoring del codice 

In [15]:
function simplifyCellsRef(V,CV)
    PRECISION = 5
    vertDict = DefaultDict{Array{Float64,1}, Int64}(0)
    index = 0
    W = Array{Float64,1}[]
    FW = Array{Int64,1}[]
    
    for incell in CV
        outcell = Int64[]
        for v in incell
         vert = V[:,v]
         key = map(approxVal(PRECISION), vert)  #key = approxVal(PRECISION) . (vert)
            if vertDict[key]==0
                index += 1
                vertDict[key] = index
                push!(outcell, index)
                push!(W,key)
            else
                push!(outcell, vertDict[key])
            end
        end
         push!(FW, [Set(outcell)...])
    end
    return hcat(W...),FW
end

simplifyCellsRef (generic function with 1 method)

In [16]:
@btime simplifyCellsRef(V,CV)

  61.965 μs (855 allocations: 81.94 KiB)


([1.0 2.0 … 66.0 62.0; 1.0 2.0 … 66.0 62.0], [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10]  …  [2], [2], [64], [20], [12], [65], [64], [25], [3], [66]])

Ho fatto un piccolo refactoring al codice della funzione principale e sembrano esserci dei miglioramenti sotto l'aspetto del tempo e in piu anche in termini di allocazione . Ho capito che usando push! al posto della pre allocazione il tempo di esecuzione aumenta del 5% che visto in termini di pochi dati è quasi nullo mentre può avere essere davvero pesante quando si parla di quantità di dati molto molto grandi . Ma in questa situazione non mi conviene sostituire i push perche come letto nella guida " If you do not know length of your array in advance, then you should use push! function." quindi ho deciso di non farlo . Invece la funzione approxVal non è stata modificata poichè è molto semplice , usata come appoggio.